# TorchScript tutorial

This tutorial will show you how to take an RNN implementation written with eager mode in mind and convert it to TorchScript code. We'll go through all the necessary steps, and see if it might give us any performance improvements (especially on the GPUs!).

Note that TorchScript is statically typed, hence we will usually need to add some `from typing` imports.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

# You might want to add those to your imports in the future!
from torch import Tensor
from typing import Tuple, Optional

The environment provided by the organizers doesn't have access to the GPUs, but Google Colab has it. This line will let us pick the fastest available device in our benchmarks.

In [ ]:
FAST_DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
FAST_DEVICE

### Everything starts with a cell
Below you can find an implementation of the basic LSTM cell. Our first task will be to convert it to a TorchScript function.

In [ ]:
def lstm_cell(input, hidden, params):
    hx, cx = hidden
    w_ih, w_hh, bias = params
    gates = torch.mm(input, w_ih.t()) + torch.mm(hx, w_hh.t()) + bias

    ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

    ingate = torch.sigmoid(ingate)
    forgetgate = torch.sigmoid(forgetgate)
    cellgate = torch.tanh(cellgate)
    outgate = torch.sigmoid(outgate)

    cy = (forgetgate * cx) + (ingate * cellgate)
    hy = outgate * torch.tanh(cy)

    return hy, cy

### Making it recurrent!

The next step will be to take the newly defined cell and make it into a proper recurrent model.

*Hint: Python 3 syntax for an argument with a type annotation and optional value is: `def f(arg : type = default):`*

In [ ]:
def init_lstm_hidden(input, params):
    # NOTE: w_ih is Tensor[hidden_size * 4, input_size]
    #       input is Tensor[seq_length, batch_size, hidden_size]
    w_ih = params[0]
    batch_size = input.size(1)
    hidden_size = w_ih.size(0) // 4
    return (torch.zeros(batch_size, hidden_size, device=input.device),
            torch.zeros(batch_size, hidden_size, device=input.device))

def sample_lstm_params(input_size, hidden_size, device=FAST_DEVICE):
    w_ih = nn.Parameter(torch.randn(hidden_size * 4, input_size, device=device, requires_grad=True))
    w_hh = nn.Parameter(torch.randn(hidden_size * 4, hidden_size, device=device, requires_grad=True))
    bias = nn.Parameter(torch.randn(hidden_size * 4, device=device, requires_grad=True))
    return w_ih, w_hh, bias

def lstm(input, params, hidden=None):
    # NOTE: input is Tensor[seq_length, batch_size, hidden_size]
    if hidden is None:
        hidden = init_lstm_hidden(input, params)
    for step_input in input.unbind():
        hidden = lstm_cell(step_input, hidden, params)
    return hidden

## Benchmarking our implementation

Now that we have our code ready, let us go through the basics of performance benchmarking and see how our model performs.

In [ ]:
import time

def benchmark(f, num_warmup=5, num_iters=100):
    synchronize = torch.cuda.synchronize if torch.cuda.is_available() else (lambda: None)
    for i in range(num_warmup):
        f()
    synchronize()
    start = time.perf_counter()
    for i in range(num_iters):
        f()
    synchronize()
    end = time.perf_counter()
    return (end - start) / num_iters

In [ ]:
def lstm_benchmark(batch_size, seq_length, input_size, hidden_size):
    input = torch.randn(seq_length, batch_size, input_size, device=FAST_DEVICE)
    params = sample_lstm_params(input_size, hidden_size, device=FAST_DEVICE)
    hidden = init_lstm_hidden(input, params)
    return benchmark(lambda: lstm(input, params, hidden), num_iters=20)
    
lstm_benchmark(batch_size=32, seq_length=64, input_size=128, hidden_size=256)

## Make all that into a module

While functions make the introduction to TorchScript simpler, in day-to-day PyTorch usage you will likely use `nn.Module`s a lot. Let's add a few lines to make a module out of our newly defined functions.

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.w_ih, self.w_hh, self.bias = sample_lstm_params(input_size, hidden_size)
        
    def forward(self, input, hidden=None):
        return lstm(input, (self.w_ih, self.w_hh, self.bias), hidden)

In [ ]:
def lstm_module_benchmark(batch_size, seq_length, input_size, hidden_size):
    lstm = LSTM(input_size, hidden_size).to(FAST_DEVICE)
    input = torch.randn(seq_length, batch_size, input_size, device=FAST_DEVICE)
    hidden = init_lstm_hidden(input, (lstm.w_ih, lstm.w_hh, lstm.bias))
    return benchmark(lambda: lstm(input, hidden))

lstm_module_benchmark(batch_size=32, seq_length=64, input_size=128, hidden_size=256)

# Exercise time!

> All exercises have a bonus optional task which is to write a benchmark function similar to `lstm_benchmark`.

## Exercise 1. Implement an MILSTM RNN.

Do the same as above, but this time implement an MILSTM cell!
This is not something that e.g. cuDNN supports.

This time we'll skip the intermediate functions for initialization/looping and implement them in the module directly.

Also, try to implement the `if hidden is None` branch using the `self.input_size` and `self.hidden_size` attributes. You'll see how it makes the code much nicer than if you inlined the body of `lstm_init_hidden`.

The equations for the cell can be found in [On Multiplicative Integration with Recurrent Neural Networks](https://arxiv.org/pdf/1606.06630.pdf), but I've also put them here for your convenience:

$$\varphi(\alpha \odot W_{ih}x \odot W_{hh}h + \beta_1 \odot W_{hh}h + \beta_2 \odot W_{ih}x + b)$$

where $\varphi$ is the usual pointwise transform used in the LSTM (i.e. the argument has to have `4 * hidden_size` features!).

In [ ]:
def milstm_cell(input, hidden, w_ih, w_hh, alpha, beta, bias):
    raise NotImplementedError

class MILSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        raise NotImplementedError
        
    def forward(self, input, hidden=None):
        raise NotImplementedError

### Exercise 2. Separate the forget-gate bias.

Remember to use `torch.jit.script`, `torch.jit.ScriptModule` and `torch.jit.script_method`!

In [ ]:
def lstm_cell_forget_bias(input, hidden, params):
    raise NotImplementedError
    
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        raise NotImplementedError
        
    def forward(self, input, hidden=None):
        if hidden is None:
            raise NotImplementedError
        raise NotImplementedError

### Exercise 3. Modify `lstm` from the first section to return intermediate hidden values

Right now only the final hidden state is returned, but in many cases (e.g. in machine translation) one actually needs all hidden states that have been produced by the network. Modify the code accordingly. See if it had any effect on the benchmark.